In [5]:
import numpy as np

class ART1:
    def __init__(self, num_features, max_clusters, vigilance=0.75):
        """
        num_features : int
            Dimensionality of each input pattern (before complement coding).
        max_clusters : int
            Maximum number of categories (nodes) the network can form.
        vigilance : float in (0,1)
            The vigilance parameter: minimum match ratio to accept a category.
        """
        self.F = num_features
        self.max_clusters = max_clusters
        self.vigilance = vigilance

        # Each weight vector is initialized to all ones (complement‐coded length = 2*F)
        self.weights = np.ones((max_clusters, 2 * num_features))

        # How many clusters have actually been “recruited” so far
        self.n_clusters = 0

    def complement_code(self, x):
        """Return [x, 1−x] concatenated along features."""
        x = np.asarray(x)
        return np.hstack([x, 1.0 - x])

    def choice_function(self, x_cc, j):
        """ART-1 choice function T_j(x) = Σ min(x, w_j)."""
        return np.sum(np.minimum(x_cc, self.weights[j]))

    def match_function(self, x_cc, j):
        """Normalized match M_j(x) = (Σ min(x, w_j)) / (Σ x)."""
        return self.choice_function(x_cc, j) / np.sum(x_cc)

    def train(self, inputs):
        """
        inputs : array-like, shape = (n_samples, num_features)
        """
        inputs = np.asarray(inputs)
        for p in inputs:
            x_cc = self.complement_code(p)
            # Try existing clusters first
            resonated = False
            for j in range(self.n_clusters):
                if self.match_function(x_cc, j) >= self.vigilance:
                    # Resonance! update weights
                    self.weights[j] = np.minimum(self.weights[j], x_cc)
                    print(f"Input {p.tolist()} → assigned to existing cluster {j}")
                    resonated = True
                    break

            if not resonated:
                # No existing cluster passed vigilance
                if self.n_clusters < self.max_clusters:
                    # Recruit a new cluster
                    j = self.n_clusters
                    self.weights[j] = x_cc.copy()
                    self.n_clusters += 1
                    print(f"Input {p.tolist()} → recruited CLUSTER {j}")
                else:
                    print(f"Input {p.tolist()} → NO MATCH and no free cluster slots")

    def predict(self, p):
        """Return the cluster index for pattern p, or -1 if none match."""
        x_cc = self.complement_code(p)
        for j in range(self.n_clusters):
            if self.match_function(x_cc, j) >= self.vigilance:
                return j
        return -1

# ——— Example usage —————————————————————————————————————————————————————
if __name__ == "__main__":
    # Toy binary patterns of length 4
    data = np.array([
        [1,0,1,0],
        [1,1,0,0],
        [0,1,1,0],
        [0,0,1,1],
        [1,0,1,0]
    ])

    art = ART1(num_features=4, max_clusters=3, vigilance=0.75)
    art.train(data)

    print("\nPredicting back the same inputs:")
    for p in data:
        c = art.predict(p)
        print(f"  {p.tolist()}  →  Cluster {c}")

Input [1, 0, 1, 0] → recruited CLUSTER 0
Input [1, 1, 0, 0] → recruited CLUSTER 1
Input [0, 1, 1, 0] → recruited CLUSTER 2
Input [0, 0, 1, 1] → NO MATCH and no free cluster slots
Input [1, 0, 1, 0] → assigned to existing cluster 0

Predicting back the same inputs:
  [1, 0, 1, 0]  →  Cluster 0
  [1, 1, 0, 0]  →  Cluster 1
  [0, 1, 1, 0]  →  Cluster 2
  [0, 0, 1, 1]  →  Cluster -1
  [1, 0, 1, 0]  →  Cluster 0
